<a href="https://colab.research.google.com/github/isisCarolineLima/ChatBot-Cidadagil/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Observação:
Esse é apenas um protótipo do chatBot para a aplicação Cidadágil. Ele ainda está sensível a erros e apenas programado para algumas situações que conseguimos idealizar. Então, até o momento o chat consegue responder sobre:  
1) Ocorrência de assalto  
2) Reserva de espaços públicos  
3) Denuncia de lixo na rua  
4) Como solicitar a poda de uma árvore  
5) Reclamação sobre o transporte público  
6) Requerimento de manutenção do sistema de iluminação público  
7) Denuncia de buracos na rua  
8) Vacinação  
9) Denuncia de maus-tratos animais  
10) Como requerir a instalação de uma lombada  
11) Como regularizar um imóvel  

In [ ]:
#Instalações necessárias
!pip install spacy
!python -m spacy download pt_core_news_md
!pip install gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 17.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import json
import spacy
import requests

# URL de exportação do Google Drive
url_banco_dados = "https://drive.google.com/uc?export=download&id=18aVHHIRTwnJM0g-PMNP0hStQUfypg_Yv"
arquivo_banco_dados = "dadosFuncionando.json"

# Baixar o banco de dados do Google Drive
def baixar_banco_dados(url, nome_arquivo):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(nome_arquivo, "wb") as arquivo:
            for chunk in response.iter_content(chunk_size=8192):
                arquivo.write(chunk)
        print("Banco de dados baixado com sucesso!")
    except requests.exceptions.RequestException as erro:
        print(f"Erro ao baixar o banco de dados: {erro}")
        exit()

# Baixar o banco de dados
baixar_banco_dados(url_banco_dados, arquivo_banco_dados)

# Carregar modelo de linguagem do spaCy
nlp = spacy.load("pt_core_news_md")

# Carregar os dados do JSON
with open(arquivo_banco_dados, "r") as arquivo:
    banco_de_dados = json.load(arquivo)

# Função para buscar a melhor resposta com base em similaridade
def buscar_resposta(situacao):
    entrada = nlp(situacao.lower())
    maior_similaridade = 0
    melhor_resposta = None

    for item in banco_de_dados:
        frase = nlp(item["situacao"].lower())
        similaridade = entrada.similarity(frase)
        if similaridade > maior_similaridade:
            maior_similaridade = similaridade
            melhor_resposta = item

    if melhor_resposta:
        return (
            f"Para {melhor_resposta['nome']} você precisa {melhor_resposta['procedimento']}.\n"
            f"Pode entrar em contato pelo telefone {melhor_resposta['contato']['telefone']}, "
            f"pelo e-mail {melhor_resposta['contato']['email']} ou visitar o endereço {melhor_resposta['contato']['endereco']}."
        )
    return "Desculpe, não encontrei informações para essa situação."

# Função para responder saudações e agradecimentos
def responder_saude_e_agradecimentos(entrada):
    entrada_lower = entrada.lower()
    if any(palavra in entrada_lower for palavra in ["bom dia", "boa tarde", "boa noite", "olá", "oi", "opa", "eae"]):
        return "Olá! Como posso ajudar você hoje?"
    if any(palavra in entrada_lower for palavra in ["tudo bem", "tudo bem?", "tudo ok", "tudo na paz", "suave", "tudo belezinha", "belezinha", "blz?"]):
        return "Eu estou ótimo! E você? Como posso ajudar você hoje?"
    return None

# Função principal do chatbot
def iniciar_chatbot():
    # Loop de conversa
    print("Olá! Sou o assistente virtual da prefeitura. Como posso ajudar hoje?")
    while True:
        entrada_usuario = input("Você: ")
        if entrada_usuario.lower() in ["sair", "encerrar", "tchau"]:
            print("Chatbot: Até logo! 😊")
            break
        elif entrada_usuario.lower() in ["obrigado", "obrigada", "valeu", "agradecido", "blz", "entendi", "vlw"]:
            print("De nada! Sempre que precisar, estarei aqui. 😊")
            break

        # Responder saudações e agradecimentos
        resposta = responder_saude_e_agradecimentos(entrada_usuario)
        if resposta:
            print(f"Chatbot: {resposta}")
            continue

        # Buscar resposta no banco de dados
        resposta = buscar_resposta(entrada_usuario)
        print(f"Chatbot: {resposta}")

# Executa o chatbot
if __name__ == "__main__":
    iniciar_chatbot()


Banco de dados baixado com sucesso!
Olá! Sou o assistente virtual da prefeitura. Como posso ajudar hoje?
Você: oi
Chatbot: Olá! Como posso ajudar você hoje?
Você: eu quero denunciar maus trtos animais
Chatbot: Para pedir regularização de imóvel você precisa verifique a documentação necessária e leve à Secretaria de Habitação para análise.
Pode entrar em contato pelo telefone 7788-6655, pelo e-mail habitacao@prefeitura.gov ou visitar o endereço Rua Moradia, 333.
Você: não maus tratos animais
Chatbot: Para denunciar maus-tratos a animais você precisa relatar o caso pelo telefone ou e-mail, informando detalhes e local para averiguação.
Pode entrar em contato pelo telefone 9988-7766, pelo e-mail meioambiente@prefeitura.gov ou visitar o endereço Av. Verde, 789.
Você: obrigada
De nada! Sempre que precisar, estarei aqui. 😊
